In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
db_path = "/content/drive/MyDrive/mydb.sqlite"
import sqlite3

conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
cur = conn.cursor()

In [3]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = [t[0] for t in cur.fetchall()]
print("Tables:", tables)

import pandas as pd
import numpy as np

for table in tables:
    print("\n" + "="*80)
    print("TABLE:", table)

    # Schema (columns)
    schema_df = pd.read_sql(f"PRAGMA table_info('{table}')", conn)
    display(schema_df)

    # Row count
    n = pd.read_sql(f"SELECT COUNT(*) AS n_rows FROM '{table}'", conn)
    display(n)

    # Sample rows
    sample_df = pd.read_sql(f"SELECT * FROM '{table}' LIMIT 5", conn)
    display(sample_df)


Tables: ['menus', 'restaurants']

TABLE: menus


,cid,name,type,notnull,dflt_value,pk
0,0,restaurant_id,INTEGER,0,None,0
1,1,category,TEXT,0,None,0
2,2,name,TEXT,0,None,0
3,3,description,TEXT,0,None,0
4,4,price,TEXT,0,None,0


,n_rows
0,5117217


,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD



TABLE: restaurants


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,0
1,1,position,INTEGER,0,None,0
2,2,name,TEXT,0,None,0
3,3,score,REAL,0,None,0
4,4,ratings,REAL,0,None,0
5,5,category,TEXT,0,None,0
6,6,price_range,TEXT,0,None,0
7,7,full_address,TEXT,0,None,0
8,8,zip_code,TEXT,0,None,0
9,9,lat,REAL,0,None,0


,n_rows
0,63469


,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",None,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640
3,4,17,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614
4,5,162,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",None,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700


In [7]:
import pandas as pd
import numpy as np
import unicodedata

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
df = restaurants.copy()

# 1) drop position column
df = df.drop(columns=["position"], errors="ignore")

# 2) clean text columns (EXCLUDING price_range)
text_cols = ["name", "category", "full_address"]

for c in text_cols:
    if c in df.columns:
        s = df[c].astype("string")

        # Unicode normalize (handles weird variants globally)
        s = s.apply(lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x)

        # Remove control / non-printable characters
        s = s.str.replace(r"[\x00-\x1F\x7F-\x9F]", "", regex=True)

        # Remove unicode "format" characters (zero-width, etc.)
        s = s.apply(
            lambda x: "".join(ch for ch in x if unicodedata.category(ch) != "Cf")
            if pd.notna(x) else x
        )

        # Collapse whitespace
        s = s.str.replace(r"\s+", " ", regex=True).str.strip()

        # Normalize missing
        s = s.replace({"": pd.NA})

        df[c] = s

# 3) clean name: remove trailing "(...)" at end
#    Example: "PJ Fresh (224 Daniel Payne Drive)" -> "PJ Fresh"
if "name" in df.columns:
    df["name"] = (
        df["name"]
        .str.replace(r"\s*\([^)]*\)\s*$", "", regex=True)
        .str.strip()
    )

# 4) score: force numeric, missing stays NaN
df["score"] = pd.to_numeric(df["score"], errors="coerce")

# 5) ratings: force numeric, missing -> 0
df["ratings"] = pd.to_numeric(df["ratings"], errors="coerce").fillna(0).astype(int)

# 6) price_range: handle SEPARATELY (no text cleaning)
if "price_range" in df.columns:
    df["price_range"] = (
        df["price_range"]
        .astype("string")
        .str.strip()
    )
    df.loc[~df["price_range"].isin(["$", "$$", "$$$", "$$$$"]), "price_range"] = pd.NA

# 7) keep category + full_address as string dtype
for c in ["category", "full_address"]:
    if c in df.columns:
        df[c] = df[c].astype("string")

# cleaned restaurants table
df.head()
len(df)


63469

In [5]:

import pandas as pd
import numpy as np
import unicodedata
menus = pd.read_sql("SELECT * FROM menus", conn)
df = menus.copy()

# 1) Clean text columns
text_cols = ["category", "name", "description"]

for c in text_cols:
    if c in df.columns:
        s = df[c].astype("string")

        # Unicode normalize
        s = s.apply(lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x)

        # Remove control / non-printable characters
        s = s.str.replace(r"[\x00-\x1F\x7F-\x9F]", "", regex=True)

        # Remove unicode format chars (zero-width, etc.)
        s = s.apply(
            lambda x: "".join(ch for ch in x if unicodedata.category(ch) != "Cf")
            if pd.notna(x) else x
        )

        # Collapse whitespace
        s = s.str.replace(r"\s+", " ", regex=True).str.strip()

        # Normalize missing
        s = s.replace({"": pd.NA})

        df[c] = s

# 2) Clean category name (light normalization)

if "category" in df.columns:
    df["category"] = df["category"].str.title()


# 3) Clean price → numeric
#    "15.99 USD" → 15.99

if "price" in df.columns:
    # extract first numeric value
    df["price_numeric"] = (
        df["price"]
        .astype("string")
        .str.extract(r"(\d+\.?\d*)")[0]
        .astype(float)
    )

    # optional: drop original USD string
    df = df.drop(columns=["price"])

# cleaned menus table
df.head()
len(df)

5117217

In [6]:
menus_clean = df.copy()


(5117217, 5)


In [8]:
restaurants_clean = df.copy()

In [9]:
print(menus_clean.shape)
print(restaurants_clean.shape)

(5117217, 5)
(63469, 10)


In [10]:
import sqlite3
import os
from google.colab import files

out_path = "/content/mydb_clean.sqlite"

# remove old file if exists
if os.path.exists(out_path):
    os.remove(out_path)

conn_out = sqlite3.connect(out_path)

# write restaurants (small)
restaurants_clean.to_sql(
    "restaurants",
    conn_out,
    if_exists="replace",
    index=False
)

# write menus (large) with chunksize
menus_clean.to_sql(
    "menus",
    conn_out,
    if_exists="replace",
    index=False,
    chunksize=100_000
)

# add indexes
cur = conn_out.cursor()
cur.execute("CREATE INDEX IF NOT EXISTS idx_restaurants_id ON restaurants(id);")
cur.execute("CREATE INDEX IF NOT EXISTS idx_menus_restaurant_id ON menus(restaurant_id);")
conn_out.commit()

# shrink db
conn_out.execute("VACUUM;")
conn_out.close()

print("Saved cleaned DB to:", out_path)

# download
files.download(out_path)


Saved cleaned DB to: /content/mydb_clean.sqlite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Data Cleaning Summary**

Loaded restaurants (63K rows) and menus (5M rows) from a SQLite database into pandas.

**Restaurants table:**

Dropped non-essential position column.

Normalized text fields (name, category, full_address) using Unicode normalization, removal of control characters, and whitespace cleanup.

Standardized restaurant names by removing trailing address fragments.

Converted score to numeric (NaN if missing/invalid).

Converted ratings to numeric and filled missing values with 0.

Cleaned price_range separately, keeping only valid $–$$$$ values.

**Menus table:**

Normalized text fields (category, name, description) using the same rule-based approach.

Extracted numeric prices from strings (e.g., "15.99 USD" → 15.99) and dropped the original text price column.

Followed a scalable, rule-based cleaning strategy with no hard-coded edge cases and no unintended row drops.